In [1]:
import pandas as pd

In [2]:
from big_picture.classifier import Classifier

SyntaxError: invalid syntax (label.py, line 109)

In [ ]:
model = Classifier()

In [ ]:
# Embedded stringe
embeddings = pd.read_csv('../big_picture/data/embeddings_200k.csv')

In [ ]:
model.fit(embeddings, printed=True, pre_made=True)

In [6]:
model